<a href="https://colab.research.google.com/github/NinaAbeyratne/Snaky---Snake-Identification-System/blob/Nina/DetectionModel_VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG19
from google.colab import drive

In [15]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [16]:
# Main directory where model data is saved
mainDataDirectory = "/content/drive/MyDrive/DSGP Group 24/Dataset/Model_Data"

In [17]:
# Categories of image classification
imageCategories = ["yes", "no"]

# Directories for training and testing data
trainDirectory = os.path.join(mainDataDirectory, 'Train')
testDirectory = os.path.join(mainDataDirectory, 'Test')

In [18]:
# Function to read images from a directory and add labels to each image
def read_images_from_directory(directory):
    images = []
    labels = []
    for category in imageCategories:
        category_path = os.path.join(directory, category)
        label = imageCategories.index(category)

        for filename in os.listdir(category_path):
            image_path = os.path.join(category_path, filename)
            image = cv2.imread(image_path)
            images.append(image)
            if label == 0:
                labels.append("no")
            elif label == 1:
                labels.append("yes")
    return images, labels

In [19]:
# Read training images
train_x, train_y = read_images_from_directory(trainDirectory)

# Read testing images
test_x, test_y = read_images_from_directory(testDirectory)

# Convert the lists to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

In [20]:
# Checking for the number of data
train_x.shape

(3984, 224, 224, 3)

In [21]:
test_x.shape

(997, 224, 224, 3)

In [22]:
# Checking the categories in the dataset
np.unique(train_y)

array(['no', 'yes'], dtype='<U3')

In [23]:
# Map string labels to integer labels inorder for the model to read the data in integer format
label_mapping = {'no': 0, 'yes': 1}

# Convert string labels to integer labels
train_y = [label_mapping[label] for label in train_y]
test_y = [label_mapping[label] for label in test_y]

# Convert labels to one-hot encoded format
train_y_one_hot = to_categorical(train_y, num_classes=2)
test_y_one_hot = to_categorical(test_y, num_classes=2)

In [24]:

# Loading the vgg16 model and setting up the image size to train
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Build the model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_x, train_y_one_hot, epochs=10, batch_size=16, validation_data=(test_x, test_y_one_hot))


80134624/80134624 [==============================] - 4s 0us/step
Epoch 1/10
249/249 [==============================] - 26s 101ms/step - loss: 2.8280 - accuracy: 0.9260 - val_loss: 0.3051 - val_accuracy: 0.9509
Epoch 2/10
249/249 [==============================] - 25s 101ms/step - loss: 0.2327 - accuracy: 0.9684 - val_loss: 0.3365 - val_accuracy: 0.9619
Epoch 3/10
249/249 [==============================] - 24s 98ms/step - loss: 0.1922 - accuracy: 0.9729 - val_loss: 0.1552 - val_accuracy: 0.9699
Epoch 4/10
249/249 [==============================] - 24s 97ms/step - loss: 0.0717 - accuracy: 0.9852 - val_loss: 0.2996 - val_accuracy: 0.9679
Epoch 5/10
249/249 [==============================] - 24s 98ms/step - loss: 0.1678 - accuracy: 0.9804 - val_loss: 0.1270 - val_accuracy: 0.9729
Epoch 6/10
249/249 [==============================] - 25s 100ms/step - loss: 0.0822 - accuracy: 0.9854 - val_loss: 0.2479 - val_accuracy: 0.9639
Epoch 7/10
249/249 [==============================] - 24s 98ms/step 

In [25]:
# Save the model
model.save('/content/drive/MyDrive/DSGP Group 24/Dataset/Model/NinaModel_VGG-19.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               6422784   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 514       
                                                                 
Total params: 26447682 (100.89 MB)
Trainable params: 6423298 (24.50 MB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________
